In [ ]:
import matplotlib.pyplot as plt
import cv2


import scipy

from pathlib import Path

import numpy as np

import h5py
import math


import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import holoviews as hv

import platform

import sys
sys.path.insert(0, "..")

from pathlib import Path

import cv2

import json


from Utilities.Utils import *
from Utilities.Processing import *

In [ ]:
# Get the DataFolder

if platform.system() == "Darwin":
    DataPath = Path("/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")
# Linux Datapath
if platform.system() == "Linux":
    DataPath = Path("/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")

print(DataPath)

In [ ]:
# Make a list of the folders I want to use
# For instance, I want to use the folders that have the "FeedingState" in the name

Folders = []
for folder in DataPath.iterdir():
    minfolder = str(folder).lower()
    if "tnt" in minfolder and "tracked" in minfolder and "pm" in minfolder:
    # Only use the folders that have 'feedingstate' and 'tracked' but not 'dark' in the name
    #if "feedingstate" in minfolder and "tracked" in minfolder and "dark" not in minfolder:
        Folders.append(folder)

Folders
    

In [ ]:
import json
import h5py
import numpy as np
import pandas as pd
from pathlib import Path

# Set the path to the data folder
data_folder = Path('/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos')

# Build a dataframe that will store the ball y positions and the arena and corridor numbers as metadata
Dataset = pd.DataFrame(columns=["Fly", "yball", "arena", "corridor", "start", "end"])

# Loop over all the .analysis.h5 files in the folder and store the ball y positions and the arena and corridor numbers as metadata
Flynum = 0
# Loop over all the folders that don't have "Dark" in the name
for folder in Folders:
    print(f"Adding experiment {folder} to the dataset...")
    # Read the metadata.json file
    with open(folder / "Metadata.json", "r") as f:
        metadata = json.load(f)
        variables = metadata["Variable"]
        metadata_dict = {}
        for var in variables:
            metadata_dict[var] = {}
            for arena in range(1, 10):
                arena_key = f"Arena{arena}"
                var_index = variables.index(var)
                metadata_dict[var][arena_key] = metadata[arena_key][var_index]
        
        print (metadata_dict)
        
    for file in folder.glob("**/*.analysis.h5"):
        #print(file)
        with h5py.File(file, "r") as f:
            dset_names = list(f.keys())
            locations = f["tracks"][:].T
            node_names = [n.decode() for n in f["node_names"][:]]

        locations.shape
        
        if "Flipped" in folder.name:
            yball[:, 0, 0] = -yball[:, 0, 0]

        else:
            yball : np.ndarray = locations[:, :, 1, :]
        
        # Get the filename from the path
        foldername = folder.name

        # Get the arena and corridor numbers from the parent (corridor) and grandparent (arena) folder names
        arena = file.parent.parent.name
        corridor = file.parent.name
        
        # Get the metadata for this arena
        arena_key = arena.capitalize()
        arena_metadata = {var: pd.Categorical([metadata_dict[var][arena_key]]) for var in metadata_dict}
        
        Flynum += 1
        
        # Load the start and end coordinates from coordinates.npy
        start, end = np.load(file.parent / 'coordinates.npy')
        
        # Store the ball y positions, start and end coordinates, and the arena and corridor numbers as metadata
        data = {"Fly": pd.Categorical(["Fly" + str(Flynum)]),
                "yball": [list(yball[:, 0, 0])], 
                "experiment": pd.Categorical([foldername]),
                "arena": pd.Categorical([arena]), 
                "corridor": pd.Categorical([corridor]),
                "start": pd.Categorical([start]),
                "end": pd.Categorical([end])}
        data.update(arena_metadata)

        # Use pandas.concat instead of DataFrame.append
        Dataset = pd.concat([Dataset, pd.DataFrame(data)], ignore_index=True) 

# Explode yball column to have one row per timepoint

#Dataset.drop(columns=["Genotye", "Date",], inplace=True)

Dataset = Dataset.explode('yball')
Dataset['yball'] = Dataset['yball'].astype(float)

print('Computing yball relative to start...')
# Compute yball_relative relative to start
Dataset['yball_relative'] = abs(Dataset['yball'] - Dataset['start'])

# Fill missing values using linear interpolation
Dataset['yball_relative'] = Dataset['yball_relative'].interpolate(method='linear')

Dataset.drop(columns=["yball",], inplace=True)


# Filter parameters
cutoff = 0.0015  # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
order = 1  # sin wave can be approx represented as quadratic

#Dataset['yball_smooth'] = butter_lowpass_filter(Dataset['yball_relative'], cutoff, order)
print('Filtering yball relative to start...')
Dataset['yball_SG'] = savgol_lowpass_filter(Dataset['yball_relative'], 221, 1)

print('Removing yball relative to start...')
Dataset.drop(columns=["yball_relative",], inplace=True)

print('Defining frame and time columns...')
Dataset["Frame"] = Dataset.groupby("Fly").cumcount()

Dataset["time"] = Dataset["Frame"] / 30

# Remove the original yball column

print('Removing Frame column...')
Dataset.drop(columns=["Frame",], inplace=True)

print('Resetting index...')
Dataset.reset_index(drop=True, inplace=True)

Dataset.head()

In [ ]:
# Filter parameters
window_length = 301
polyorder = 1

Dataset['yball_SG'] = savgol_lowpass_filter(Dataset['yball_relative'], 93, 1)
Dataset['yball_SG2'] = savgol_lowpass_filter(Dataset['yball_relative'], 151, 1)
Dataset['yball_SG3'] = savgol_lowpass_filter(Dataset['yball_relative'], 191, 1)
Dataset['yball_SG4'] = savgol_lowpass_filter(Dataset['yball_relative'], 201, 1)
Dataset['yball_SG5'] = savgol_lowpass_filter(Dataset['yball_relative'], 221, 1) # This is the one i'll go with for now
# Filter parameters
# cutoff = 0.01  # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
# order = 1 
# Dataset['yball_SG_butter'] = butter_lowpass_filter(Dataset['yball_SG'], cutoff, order)


In [ ]:
mse_SG = ((Dataset['yball_SG'] - Dataset['yball_relative']) ** 2).mean()
mse_SG2 = ((Dataset['yball_SG2'] - Dataset['yball_relative']) ** 2).mean()
mse_SG3 = ((Dataset['yball_SG3'] - Dataset['yball_relative']) ** 2).mean()
mse_SG4 = ((Dataset['yball_SG4'] - Dataset['yball_relative']) ** 2).mean()
mse_SG5 = ((Dataset['yball_SG5'] - Dataset['yball_relative']) ** 2).mean()

print(mse_SG, mse_SG2, mse_SG3, mse_SG4, mse_SG5)

In [ ]:
from scipy import stats
from sklearn.metrics import mutual_info_score, r2_score
import matplotlib.pyplot as plt

window_lengths = range(3, 103, 10)
mae_values = []
r2_values = []
mi_values = []

for window_length in window_lengths:
    # Apply the Savitzky-Golay filter with the current window_length
    y = savgol_lowpass_filter(Dataset['yball_relative'], window_length, polyorder=1)
    
    # Compute the MAE between the filtered signal and the raw data
    mae = np.abs(y - Dataset['yball_relative']).mean()
    mae_values.append(mae)
    
    # Compute the R-squared between the filtered signal and the raw data
    r2 = r2_score(Dataset['yball_relative'], y)
    r2_values.append(r2)
    
    # Compute the mutual information between the filtered signal and the raw data
    mi = mutual_info_score(y, Dataset['yball_relative'])
    mi_values.append(mi)

# Plot the results
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
axs[0].plot(window_lengths, mae_values)
axs[0].set_title('MAE')
axs[1].plot(window_lengths, r2_values)
axs[1].set_title('R-squared')
axs[2].plot(window_lengths, mi_values)
axs[2].set_title('Mutual Information')
plt.show()


In [ ]:
# Find the index of the minimum MAE value
best_mae_index = np.argmin(mae_values)
# Find the corresponding window_length value
best_mae_window_length = window_lengths[best_mae_index]

# Find the index of the maximum R-squared value
best_r2_index = np.argmax(r2_values)
# Find the corresponding window_length value
best_r2_window_length = window_lengths[best_r2_index]

# Find the index of the maximum mutual information value
best_mi_index = np.argmax(mi_values)
# Find the corresponding window_length value
best_mi_window_length = window_lengths[best_mi_index]

print(f'Best MAE: {best_mae_window_length}')
print(f'Best R-squared: {best_r2_window_length}')
print(f'Best Mutual Information: {best_mi_window_length}')


In [ ]:
noise_estimate = np.std(Dataset['yball_relative'] - Dataset['yball_SG'])
snr_SG = 10 * np.log10(np.mean(Dataset['yball_SG'] ** 2) / noise_estimate ** 2)

noise_estimate = np.std(Dataset['yball_relative'] - Dataset['yball_SG2'])
snr_SG2 = 10 * np.log10(np.mean(Dataset['yball_SG2'] ** 2) / noise_estimate ** 2)

noise_estimate = np.std(Dataset['yball_relative'] - Dataset['yball_SG3'])
snr_SG3 = 10 * np.log10(np.mean(Dataset['yball_SG3'] ** 2) / noise_estimate ** 2)

noise_estimate = np.std(Dataset['yball_relative'] - Dataset['yball_SG4'])
snr_SG4 = 10 * np.log10(np.mean(Dataset['yball_SG4'] ** 2) / noise_estimate ** 2)

noise_estimate = np.std(Dataset['yball_relative'] - Dataset['yball_SG5'])
snr_SG5 = 10 * np.log10(np.mean(Dataset['yball_SG5'] ** 2) / noise_estimate ** 2)

print(snr_SG, snr_SG2, snr_SG3, snr_SG4, snr_SG5)

Note on filtering: 



AL


In [ ]:
# Filter parameters
cutoff = 0.01  # desired cutoff frequency of the filter, Hz
order = 1  # sin wave can be approx represented as quadratic
rp = 2  # maximum ripple allowed in passband (dB)

Dataset['yball_C'] = cheby1_lowpass_filter(Dataset['yball_relative'], cutoff, order, rp)


In [ ]:
# Filter parameters
cutoff = 0.0015  # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
order = 1  # sin wave can be approx represented as quadratic



Dataset['yball_smooth'] = butter_lowpass_filter(Dataset['yball_relative'], cutoff, order)

In [ ]:
from scipy import signal

def median_filter(data, kernel_size):
    # Apply the median filter
    y = signal.medfilt(data, kernel_size)
    return y



In [ ]:
from scipy import ndimage

def morphological_filter(data, structure):
    # Apply the morphological filter
    y = ndimage.grey_opening(data, structure=structure)
    return y


In [ ]:
from scipy import signal
import numpy as np

from scipy import signal
import numpy as np

def adaptive_moving_average_filter(data, cutoff, order, window_size):
    # Apply the Butterworth low-pass filter
    y = butter_lowpass_filter(data, cutoff, order)
    # Compute the local standard deviation of the signal
    std = np.std(y)
    # Compute the adaptive moving average window size
    window = int(np.maximum(1, np.minimum(window_size, np.median(std / np.abs(y)))))
    # Apply the adaptive moving average filter
    y = np.convolve(y, np.ones(window) / window, mode='same')
    return y


# Filter parameters
cutoff = 0.005  # desired cutoff frequency of the Butterworth filter, Hz
order = 1  # sin wave can be approx represented as quadratic
window_size = 10  # maximum size of the moving average window

Dataset['yball_smooth_adapt'] = adaptive_moving_average_filter(Dataset['yball_relative'], cutoff, order, window_size)


In [ ]:
# Median filter parameters
kernel_size = 21

Dataset['yball_smooth_median'] = median_filter(Dataset['yball_relative'], kernel_size)

# Morphological filter parameters
structure = np.ones(21)

Dataset['yball_smooth_morph'] = morphological_filter(Dataset['yball_relative'], structure)


In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

# Select the flies to plot
flies_to_plot = ['Fly58']

# Create an empty overlay
overlay = hv.Overlay()

# Plot the data for each fly
for fly in flies_to_plot:
    fly_data = Dataset[Dataset['Fly'] == fly]
    yball_relative_curve = hv.Curve((fly_data['Frame'], fly_data['yball_relative']), label=f'{fly} yball_relative')
    yball_smoothed_curve = hv.Curve((fly_data['Frame'], fly_data['yball_smooth']), label=f'{fly} yball_smooth')
    # yball_smoothed_adapt = hv.Curve((fly_data['Frame'], fly_data['yball_smooth_adapt']), label=f'{fly} yball_smooth_adapt')
    yball_sg_curve = hv.Curve((fly_data['Frame'], fly_data['yball_SG']), label=f'{fly} yball_SG')
    # yball_sg2_curve = hv.Curve((fly_data['Frame'], fly_data['yball_SG2']), label=f'{fly} yball_SG2')
    # yball_sg3_curve = hv.Curve((fly_data['Frame'], fly_data['yball_SG3']), label=f'{fly} yball_SG3')
    # yball_sg4_curve = hv.Curve((fly_data['Frame'], fly_data['yball_SG4']), label=f'{fly} yball_SG4')
    # yball_sg5_curve = hv.Curve((fly_data['Frame'], fly_data['yball_SG5']), label=f'{fly} yball_SG5')
    #yball_sg_butter = hv.Curve((fly_data['Frame'], fly_data['yball_SG_butter']), label=f'{fly} yball_SG_butter')

    #yball_smoothed_median = hv.Curve((fly_data['Frame'], fly_data['yball_smooth_median']), label=f'{fly} yball_smooth_median')
    #yball_smoothed_morph = hv.Curve((fly_data['Frame'], fly_data['yball_smooth_morph']), label=f'{fly} yball_smooth_morph')
    overlay *= yball_relative_curve * yball_sg_curve  * yball_smoothed_curve

# Set the options for the plot
overlay.opts(
    opts.Curve(width=1500, height=1000, tools=['hover', 'box_zoom'], line_width=3)
)

# Show the plot
overlay



In [ ]:
# Display data head for these flies
Dataset[Dataset['Fly'].isin(flies_to_plot)].head()

In [ ]:
# Fill missing values using linear interpolation
Dataset['yball_relative'] = Dataset['yball_relative'].interpolate(method='linear')

# Filter parameters
cutoff = 0.01  # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
order = 1  # sin wave can be approx represented as quadratic

Dataset['yball_smooth'] = butter_lowpass_filter(Dataset['yball_relative'], cutoff, order)

Dataset.head()

In [ ]:
# Select the columns that we want to keep
Dataset = Dataset[["Fly", "Genotype", "time", "yball_relative", "yball_smooth", "yball_SG"]]
Dataset.head()

In [ ]:
# Replace all occurrences of "Fed" with "fed" in the 'FeedingState' column
Dataset["FeedingState"] = Dataset["FeedingState"].replace("Fed", "fed")

# Add "flipped" to the list of categories for the Orientation column
Dataset['Orientation'] = Dataset['Orientation'].cat.add_categories(['flipped'])

# If there is 'Flipped' in the foldername, replace the corresponding 'Orientation' with 'flipped'
Dataset.loc[Dataset["experiment"].str.contains("Flipped"), "Orientation"] = "flipped"


In [ ]:
GroupedDF_TNT = Dataset.groupby(['Genotype','time',])['yball_SG'].mean().reset_index()

In [ ]:
# Calculate the sample size for each Genotype group based on the number of individual flies
sample_size = Dataset.groupby('Genotype')['Fly'].nunique()

# Set the figure size
plt.figure(figsize=(12, 6))

# Create a line plot of the 'yball' column over time, colored by the 'Genotype' column
sns.lineplot(data=GroupedDF_TNT, x='time', y='yball_SG', hue='Genotype', linewidth=1)

# Invert the y-axis
#plt.gca().invert_yaxis()

# Modify the labels of the legend to include the sample size for each Genotype group
legend = plt.legend()
for text, genotype in zip(legend.texts, sample_size.index):
    text.set_text(f'{genotype} (n = {sample_size[genotype]})')

# Show the plot
plt.show()

In [ ]:
# Compute the difference between consecutive yball_relative values for each fly
Dataset['yball_relative_diff'] = Dataset.groupby('Fly')['yball_SG'].diff()

# Compute the cumulative_push and cumulative_pull for each fly
Dataset['cumulative_push'] = Dataset.apply(lambda x: x['yball_relative_diff'] if x['yball_relative_diff'] > 0 else 0, axis=1).groupby(Dataset['Fly']).cumsum()
Dataset['cumulative_pull'] = Dataset.apply(lambda x: -x['yball_relative_diff'] if x['yball_relative_diff'] < 0 else 0, axis=1).groupby(Dataset['Fly']).cumsum()


In [ ]:
# Group the data by Genotype and time and compute the mean of the cumulative_push and cumulative_pull columns
GroupedDF = Dataset.groupby(['Genotype', 'time'])[['cumulative_push', 'cumulative_pull']].mean().reset_index()

In [ ]:
# Plot the cumulative_push and cumulative_pull columns over time, colored by the Genotype column in two separate subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
sns.lineplot(data=GroupedDF, x='time', y='cumulative_push', hue='Genotype', linewidth=1, ax=axes[0])
sns.lineplot(data=GroupedDF, x='time', y='cumulative_pull', hue='Genotype', linewidth=1, ax=axes[1])


In [ ]:
Dataset.head()

In [ ]:
# Plot cumulative_push and cumulative_pull over time for 3 randomly selected flies in 3 subplots
fig, axes = plt.subplots(3, 1, figsize=(12, 6), sharex=True)
for ax, fly in zip(axes, np.random.choice(Dataset['Fly'].unique(), 3)):
    sns.lineplot(data=Dataset[Dataset['Fly'] == fly], x='time', y='cumulative_push', hue='Genotype', linewidth=1, ax=ax)
    sns.lineplot(data=Dataset[Dataset['Fly'] == fly], x='time', y='cumulative_pull', hue='Genotype', linewidth=1, ax=ax)
    

In [ ]:
# plot the yball_relative_diff column for each fly over time each in their separate subplot, colored by the Genotype column
g = sns.FacetGrid(data=Dataset, col='Fly', col_wrap=4, hue='Genotype', sharey=False, height=3, aspect=1.5)
g.map(sns.lineplot, 'time', 'yball_relative', linewidth=1)
g.set_titles('{col_name}')
g.set_axis_labels('Time (s)', 'yball_relative')
g.add_legend()

In [ ]:
# Replace all occurrences of "Fed" with "fed" in the 'FeedingState' column
Dataset["FeedingState"] = Dataset["FeedingState"].replace("Fed", "fed")

# Add "flipped" to the list of categories for the Orientation column
Dataset['Orientation'] = Dataset['Orientation'].cat.add_categories(['flipped'])

# If there is 'Flipped' in the foldername, replace the corresponding 'Orientation' with 'flipped'
Dataset.loc[Dataset["experiment"].str.contains("Flipped"), "Orientation"] = "flipped"

In [ ]:
GroupedDF = Dataset.groupby(['Period','FeedingState', 'time'])['yball_SG'].mean().reset_index()

In [ ]:
# Calculate the sample size for each Genotype group based on the number of individual flies
sample_size = Dataset.groupby('Period','FeedingState')['Fly'].nunique()

# Set the figure size
plt.figure(figsize=(12, 6))

# Create a line plot of the 'yball' column over time, colored by the 'Genotype' column
sns.lineplot(data=GroupedDF_TNT, x='time', y='yball_SG', hue='Genotype', linewidth=1)

# Invert the y-axis
#plt.gca().invert_yaxis()

# Modify the labels of the legend to include the sample size for each Genotype group
legend = plt.legend()
for text, genotype in zip(legend.texts, sample_size.index):
    text.set_text(f'{genotype} (n = {sample_size[genotype]})')

# Show the plot
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a FacetGrid object with the 'Period' column as the row variable
g = sns.FacetGrid(data=GroupedDF, row='Period')

# Map a line plot of the 'yball_SG' column over time to each facet, with the hue set to 'FeedingState'
g.map(sns.lineplot, 'time', 'yball_SG', 'FeedingState', linewidth=1)

# Add a legend to the plot
sample_size = Dataset.groupby(['Period','FeedingState'])['Fly'].nunique()
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
for text, genotype in zip(legend.texts, sample_size.index):
    text.set_text(f'{genotype} (n = {sample_size[genotype]})')

# Invert the y-axis of each Axes object in the FacetGrid
for ax in g.axes.flat:
    ax.invert_yaxis()

# Show the plot
plt.show()


In [ ]:
sample_size = Dataset.groupby(['Period','FeedingState'])['Fly'].nunique()
print(sample_size)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define data
data = GroupedDF

# Create a list of periods
periods = data['Period'].unique()

# Create a figure with two subplots, one for each period
fig, axes = plt.subplots(nrows=len(periods), figsize=(8, 6))

# Set the same x and y limits for both subplots
axes[0].set_xlim(data['time'].min(), data['time'].max())
axes[0].set_ylim(data['yball_SG'].min(), data['yball_SG'].max())
axes[1].set_xlim(data['time'].min(), data['time'].max())
axes[1].set_ylim(data['yball_SG'].min(), data['yball_SG'].max())

# Iterate over the periods and create a plot for each one
for i, period in enumerate(periods):
    # Subset the data by period
    data_period = data[data['Period'] == period]
    
    # Create a line plot of 'yball_SG' over time, with the hue set to 'FeedingState'
    sns.lineplot(data=data_period, x='time', y='yball_SG', hue='FeedingState', linewidth=1, ax=axes[i])
    
    # Set the title of the subplot
    axes[i].set_title(period)
    
    # Invert the y-axis of the subplot
    #axes[i].invert_yaxis()
    
    # Add a legend to the subplot
    sample_size = Dataset.groupby(['Period','FeedingState'])['Fly'].nunique()
    legend = axes[i].legend(title='FeedingState', bbox_to_anchor=(1.05, 1), loc='upper left')
    for text, genotype in zip(legend.texts[1:], sample_size.loc[period].index):
        text.set_text(f'{genotype} (n = {sample_size.loc[period, genotype]})')

# Adjust the layout of the subplots
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
data_am.head()

In [ ]:
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')

# Define data
data = GroupedDF

# Create a dataset
ds = hv.Dataset(data, kdims=['time', 'Period'], vdims=['yball_SG', 'FeedingState'])

# Create a grid of line plots
grid = ds.to(hv.Curve, 'time', 'yball_SG', groupby=['Period', 'FeedingState']).opts(
    opts.Curve(width=600, height=400, invert_yaxis=True, show_legend=True))

# Add sample size to legend
sample_size = Dataset.groupby(['Period','FeedingState'])['Fly'].nunique()
#grid = grid.opts(opts.Curve(legend_limit=len(sample_size)))
for i, genotype in enumerate(sample_size.index):
    grid = grid.opts(opts.Curve(legend_title=f'{genotype} (n = {sample_size[genotype]})', #legend_position='right',
                                show_legend=True), index=i)

# Display the plot
grid


In [ ]:
import hvplot.pandas

# Define data
data = GroupedDF

# Create a grid of line plots
grid = data.hvplot.line(
    x='time', y='yball_SG', by='FeedingState', subplots=True, row='Period',
    width=600, height=400, invert_yaxis=True, legend='top_right'
)

# Add sample size to legend
sample_size = Dataset.groupby(['Period','FeedingState'])['Fly'].nunique()
for i, genotype in enumerate(sample_size.index):
    grid[i].legend.title = f'{genotype} (n = {sample_size[genotype]})'

# Display the plot
grid


In [ ]:
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')

# Define data
data = GroupedDF

# Create a dataset
ds = hv.Dataset(data, kdims=['time', 'Period'], vdims=['yball_SG', 'FeedingState'])

# Create a list of curves
curves = []
for period in data['Period'].unique():
    for feeding_state in data['FeedingState'].unique():
        curve = ds.select(Period=period, FeedingState=feeding_state).to(hv.Curve, 'time', 'yball_SG')
        curve = curve.relabel(f'{period} - {feeding_state}')
        curves.append(curve)

# Overlay the curves
overlay = hv.Overlay(curves).opts(
    opts.Curve(width=600, height=400, show_legend=True))

# Display the plot
overlay
